# Chapter 19 Exercises (Non Coding)

1. What does a SavedModel contain? How do you inspect its content?
2. When should you use TF Serving? What are its main features? What are some
tools you can use to deploy it?
3. How do you deploy a model across multiple TF Serving instances?
4. When should you use the gRPC API rather than the REST API to query a model
served by TF Serving?
5. What are the different ways TFLite reduces a model’s size to make it run on a
mobile or embedded device?
6. What is quantization-aware training, and why would you need it?
7. What are model parallelism and data parallelism? Why is the latter generally
recommended?
8. When training a model across multiple servers, what distribution strategies can
you use? How do you choose which one to use?

***1. What does a SavedModel contain? How do you inspect its content?***

A: A savedmodel is a better way of saving TF models. It contains a TF model with the architecture and weights. This is stored instead of an h5 file and models with huge architectures can be split across multiple files. You can inspect a modelfile by using tf.saved_model.load()

***2. When should you use TF Serving? What are its main features? What are some
tools you can use to deploy it?***

A: TF Serving is useful for deploying models using a REST API or gRPC API. It should be useed when you want these models directly placed into applications in a workplace. There are many useful features such as autodeploying models, testing with experimental models, and similar to other code repos like TFS or Git, you can deploy new model versions and revert changes if needed. If you implement Docker images to your application, you could also build it under a host solution like AWS or Google Cloud. 

***3. How do you deploy a model across multiple TF Serving instances?***

A: This is done by configuring the TF Serving instances to monitor the same models directory. Then it's as simple as exporting the model as a SavedModel file into the subdirectory of your choosing. 

***4. When should you use the gRPC API rather than the REST API to query a model
served by TF Serving?***

A: gRPC API would be better for the best possible performance needed typically because it is known to be more efficient than the REST API. 

***5. What are the different ways TFLite reduces a model’s size to make it run on a
mobile or embedded device?***

A: There is a converter than shrinks the model sizes and gets rid of any operations that aren't needed for predictive features. Additionally, there is a flatbuffer format that can be used by the RAM which assists with loading times and memory allocation. 

***6. What is quantization-aware training, and why would you need it?***

A: Quant-aware training simulates low precision behavior in the forward pass, while the backward pass remains the same. This induces some quantization error which is accumulated in the total loss of the model and hence the optimizer tries to reduce it by adjusting the parameters accordingly. This makes the parameters more robust to quantization making the process almost lossless. This helps with the final weights and makes them more robust. 

***7. What are model parallelism and data parallelism? Why is the latter generally
recommended?

A: Model parallelism is running a model across multiple GPUs or CPUs. Data parallelism is just grabbing the same model and deploying it on several devices with new data batches from training/testing data and then computes the associated losses for the gradients. Data parallelism is generally better for faster training and for ease of implementation as well because of how many ways the model is chopped across the devices. 

***8. When training a model across multiple servers, what distribution strategies can
you use? How do you choose which one to use?***

A: 
- The *MultiWorkerMirroredStrategy* performs mirrored data parallelism. The
model is replicated across all available servers and devices, and each replica
gets a different batch of data at each training iteration and computes its own
gradients. The mean of the gradients is computed and shared across all replicas
using a distributed AllReduce implementation (NCCL by default), and all rep‐
licas perform the same Gradient Descent step. This strategy is the simplest to
use since all servers and devices are treated in exactly the same way, and it per‐
forms fairly well. In general, you should use this strategy. Its main limitation is
that it requires the model to fit in RAM on every replica.

- The *ParameterServerStrategy* performs asynchronous data parallelism. The
model is replicated across all devices on all workers, and the parameters are
sharded across all parameter servers. Each worker has its own training loop,
running asynchronously with the other workers; at each training iteration,
each worker gets its own batch of data and fetches the latest version of the
model parameters from the parameter servers, then it computes the gradients
of the loss with regard to these parameters, and it sends them to the parameter
servers. Lastly, the parameter servers perform a Gradient Descent step using
these gradients. This strategy is generally slower than the previous strategy,
and a bit harder to deploy, since it requires managing parameter servers. How‐
ever, it is useful to train huge models that don’t fit in GPU RAM.